## Ratings Projections (Linear Model and Matrix Factorization)

In [2]:
import csv

import numpy as np
from sklearn.metrics import mean_squared_error as mse

from tqdm import tqdm_notebook as tqdm

In [3]:
import numpy as np
import pandas as pd
import gzip
from collections import defaultdict

def readGz(f):
    for l in gzip.open(f):
        yield eval(l)

In [4]:
import shutil
with open(r'\Users\Julie\Documents\Andrew MBA stuff\Machine Learning\Kaggle Project\Rating\train.json', 'rb') as f_in:
    with gzip.open('train.json.gz', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [5]:
import pandas as pd
import gzip

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

df = getDF('train.json.gz')

In [6]:
df

,reviewTime,reviewText,helpful,reviewerID,reviewHash,categories,unixReviewTime,itemID,rating,summary,categoryID,price
0,"09 26, 2013",The model in this picture has them rolled up a...,"{'nHelpful': 0, 'outOf': 0}",U490934656,R798569390,"[[Clothing, Shoes & Jewelry, Women], [Clothing...",1380153600,I402344648,4.0,High Waisted,0,NaN
1,"02 7, 2013","I love the look of this bra, it is what I want...","{'nHelpful': 0, 'outOf': 0}",U714157797,R436443063,"[[Clothing, Shoes & Jewelry, Women, Clothing, ...",1360195200,I697650540,4.0,Beautiful but size runs small,0,NaN
2,"03 16, 2014",I am not comfortable with wearing my wedding b...,"{'nHelpful': 0, 'outOf': 0}",U507366950,R103439446,"[[Clothing, Shoes & Jewelry, Wedding Party Gif...",1394928000,I464613034,5.0,Great Alternative for Nurses,0,19.99
3,"03 10, 2014",Like the look of this top and really looks cut...,"{'nHelpful': 0, 'outOf': 0}",U307862152,R486351639,"[[Clothing, Shoes & Jewelry, Women, Clothing, ...",1394409600,I559560885,2.0,One size fits all...Questionable,0,18.99
4,"07 30, 2013",I'm quite small and the XS fits me like a regu...,"{'nHelpful': 1, 'outOf': 1}",U742726598,R508664275,"[[Clothing, Shoes & Jewelry, Women, Plus-Size,...",1375142400,I476005312,5.0,Great shirt,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
199995,"01 26, 2013",Looks just like the picture and the item arriv...,"{'nHelpful': 0, 'outOf': 0}",U781794983,R285432298,"[[Clothing, Shoes & Jewelry, Women, Clothing, ...",1359158400,I245323432,5.0,Cute,0,NaN
199996,"04 2, 2013",I'm a C cup and the C cup in this is way too s...,"{'nHelpful': 1, 'outOf': 1}",U151975942,R454986738,"[[Clothing, Shoes & Jewelry, Women, Clothing, ...",1364860800,I990230316,1.0,Not true to Cup size.,0,NaN
199997,"09 22, 2012",I have a pair of earrings just like this (only...,"{'nHelpful': 0, 'outOf': 0}",U525354881,R088851171,"[[Clothing, Shoes & Jewelry, Jewelry: Internat...",1348272000,I037381245,4.0,"Lovely, but small",0,34.99
199998,"12 27, 2013",My granddaughter love the scarf and fit perfec...,"{'nHelpful': 0, 'outOf': 1}",U995566285,R524991477,"[[Clothing, Shoes & Jewelry, Women, Accessorie...",1388102400,I343675670,5.0,Scarf,0,7.50


In [7]:
import string

input_data = df
input_data['reviewerID'] = input_data['reviewerID'].astype(object) # fix datatype error
input_data['itemID'] = input_data['itemID'].astype(object) # fix datatype error
input_data['rating'] = input_data['rating'].astype(object) # fix datatype error

dataset = {"reviewerID": input_data["reviewerID"], "itemID": input_data["itemID"], "rating": input_data["rating"]  }
dataset = pd.DataFrame(data = dataset)
dataset = dataset.dropna()

# dataset = dataset[dataset["overall"] != '3'] # need datatype=object
#dataset["label"] = dataset["rating"]
dataset

,reviewerID,itemID,rating
0,U490934656,I402344648,4
1,U714157797,I697650540,4
2,U507366950,I464613034,5
3,U307862152,I559560885,2
4,U742726598,I476005312,5
...,...,...,...
199995,U781794983,I245323432,5
199996,U151975942,I990230316,1
199997,U525354881,I037381245,4
199998,U995566285,I343675670,5


In [8]:
def readFile(f):
    for l in open(f):
        yield eval(l)

In [9]:
dataset['reviewerID']=dataset['reviewerID'].str[1:]
dataset['itemID']=dataset['itemID'].str[1:]

In [10]:
dataset

,reviewerID,itemID,rating
0,490934656,402344648,4
1,714157797,697650540,4
2,507366950,464613034,5
3,307862152,559560885,2
4,742726598,476005312,5
...,...,...,...
199995,781794983,245323432,5
199996,151975942,990230316,1
199997,525354881,037381245,4
199998,995566285,343675670,5


In [ ]:
dataset.to_csv(r'\Users\Julie\Documents\Andrew MBA stuff\Machine Learning\Kaggle Project\Rating\train_ratings.csv')

## Export and Edit CSV FILE!!!

## Start of Linear Model

In [ ]:
total_sample = 200000
train_sample = 150000

allRatings = np.zeros(total_sample)   # store all ratings
train_user_diction = {}   # store data using userID as keys
train_item_diction = {}   # store data using itemID as keys
test_user_diction = {}   # store data using userID as keys
test_item_diction = {}   # store data using itemID as keys
data = np.zeros((total_sample, 3), dtype=object) # store entire data as an array


reader = csv.reader(open(r"\Users\Julie\Documents\Andrew MBA stuff\Machine Learning\Kaggle Project\Rating\train_ratings.csv", 'r', encoding = "ISO-8859-1"), delimiter=',')
next(reader) # to skip the first line

for index, row in enumerate(reader):
    allRatings[index] = float(row[-1])
    data[index] = row
    if index < train_sample:
        if row[0] not in train_user_diction:
            train_user_diction[row[0]] = [row[1:]]
        else:
            train_user_diction[row[0]].append(row[1:])
        if row[1] not in train_item_diction:
            train_item_diction[row[1]] = [[row[0], row[2]]]
        else:
            train_item_diction[row[1]].append([row[0], row[2]])
    else:
        if row[0] not in test_user_diction:
            test_user_diction[row[0]] = [row[1:]]
        else:
            test_user_diction[row[0]].append(row[1:])
        if row[1] not in test_item_diction:
            test_item_diction[row[1]] = [[row[0], row[2]]]
        else:
            test_item_diction[row[1]].append([row[0], row[2]])


In [ ]:
data

In [ ]:
# model 1: only alpha
alpha = np.mean(allRatings[:train_sample])/1.00205
test_error = mse(allRatings[train_sample:], [alpha]*(total_sample - train_sample))
train_error = mse(allRatings[:train_sample], [alpha]*train_sample)
print('model 1 alpha', alpha) 
print('model 1 train mse', train_error)
print('model 1 train rmse', np.sqrt(train_error))
print('model 1 testing mse', test_error)
print('model 1 testing rmse', np.sqrt(test_error))

In [ ]:
# model 2: alpha and beta_u

# initialize alpha and beta
train_user_list = list(train_user_diction.keys())
alpha_2 = 0
beta_u = {}
delta = 10  # difference between iterations
while delta >= 1000**-5: # set the error tolerance 
    local_alpha = 0
    for user in train_user_list:   # we first update alpha
        user_rating = np.array(train_user_diction[user])[:,1]        
        if user not in beta_u:
            local_alpha += np.sum(user_rating.astype(float))
        else:
            local_alpha += np.sum(user_rating.astype(float) - beta_u[user])/2.5
    local_alpha = local_alpha/train_sample
    
    for user in train_user_list:  # we then update beta
        user_rating = np.array(train_user_diction[user])[:,1]
        beta_u[user] = np.sum(user_rating.astype(float) - local_alpha)/len(train_user_diction[user])
    
    delta = abs(local_alpha - alpha_2)  # calculate the difference of alphas between epoch
    alpha_2 = local_alpha # update global alpha
    print(alpha_2)

print(alpha_2)

##### calculate training mse and testing mse #######################################################
test_user_list = list(test_user_diction.keys())
train_label = np.zeros(train_sample)
train_prediction = np.zeros(train_sample)
test_label = np.zeros(total_sample-train_sample)
test_prediction = np.zeros(total_sample - train_sample)
index = 0
for user in train_user_list:
    user_rating = np.array(train_user_diction[user])[:,1].astype(float)
    train_label[index: index+len(user_rating)] = user_rating
    train_prediction[index: index+len(user_rating)] = alpha_2 + beta_u[user]
    index += len(user_rating)

index = 0
for user in test_user_list:
    user_rating = np.array(test_user_diction[user])[:,1].astype(float)
    test_label[index: index+len(user_rating)] = user_rating
    if user not in train_user_list:
        test_prediction[index: index+len(user_rating)] = alpha_2
    else:
        test_prediction[index: index+len(user_rating)] = alpha_2 + beta_u[user]
    index += len(user_rating)
        
train_error_2 = mse(train_label, train_prediction)
test_error_2 = mse(test_label, test_prediction)
print('model 2 train mse', train_error_2)
print('model 2 train rmse', np.sqrt(train_error_2))
print('model 2 testing mse', test_error_2)
print('model 2 testing rmse', np.sqrt(test_error_2))

In [ ]:
train_user_diction[user]

In [ ]:
# model 3: alpha and beta_i

# initialize alpha and beta
train_item_list = list(train_item_diction.keys())
alpha_3 = 0
beta_i = {}
delta = 10
while delta >= 10**-5:
    local_alpha = 0
    for item in train_item_list: # we first update alpha
        item_rating = np.array(train_item_diction[item])[:,1]
        if item not in beta_i:
            local_alpha += np.sum(item_rating.astype(float))
        else:
            local_alpha += np.sum(item_rating.astype(float) - beta_i[item])
    local_alpha = local_alpha/train_sample
    
    for item in train_item_list: # we then update beta
        item_rating = np.array(train_item_diction[item])[:,1]
        beta_i[item] = np.sum(item_rating.astype(float) - local_alpha)/len(train_item_diction[item])
    
    delta = abs(local_alpha - alpha_3)
    alpha_3 = local_alpha
    print(alpha_3)


test_item_list = list(test_item_diction.keys())
train_label = np.zeros(train_sample)
train_prediction = np.zeros(train_sample)
test_label = np.zeros(total_sample - train_sample)
test_prediction = np.zeros(total_sample - train_sample)
index = 0
for item in tqdm(train_item_list, total = len(train_item_list)):
    item_rating = np.array(train_item_diction[item])[:,1].astype(float)
    train_label[index: index+len(item_rating)] = item_rating
    train_prediction[index: index+len(item_rating)] = alpha_3 + beta_i[item]/2
    index += len(item_rating)

index = 0
for item in tqdm(test_item_list, total = len(test_item_list)):
    item_rating = np.array(test_item_diction[item])[:,1].astype(float)
    test_label[index: index+len(item_rating)] = item_rating
    if item not in train_item_list:
        test_prediction[index: index+len(item_rating)] = alpha_3
    else:
        test_prediction[index: index+len(item_rating)] = alpha_3 + beta_i[item]/2
    index += len(item_rating)
        
train_error_3 = mse(train_label, train_prediction)
test_error_3 = mse(test_label, test_prediction)
print('model 3 train mse', train_error_3)
print('model 3 train rmse', np.sqrt(train_error_3))
print('model 3 testing mse', test_error_3)
print('model 3 testing rmse', np.sqrt(test_error_3))

In [ ]:
 data[:train_sample]

In [ ]:
# model 4: alpha and beta_u and beta_i

# initialize alpha and beta
alpha_4 = 0
beta_u = {}
beta_i = {}
delta = 10
epoch = 0
while delta >= 10**-12:
    local_alpha = 0
    local_beta_u = {}
    local_beta_i = {}
    
    for user, item, rating in data[:train_sample]:  # we first update alpah
        if user not in beta_u:
            if item not in beta_i:
                local_alpha += float(rating)/train_sample
            else:
                local_alpha += (float(rating) - beta_i[item])/train_sample
        else:
            if item not in beta_i:
                local_alpha += (float(rating) - beta_u[user])/train_sample
            else:
                local_alpha += (float(rating) - beta_u[user] - beta_i[item])/train_sample
    delta = abs(alpha_4 - local_alpha)
    alpha_4 = local_alpha
    print(alpha_4)
    
    for user, item, rating in data[:train_sample]:  # we first update beta_u
        if item not in beta_i:
            if user not in local_beta_u:
                local_beta_u[user] = (float(rating) - alpha_4)/len(train_user_diction[user])
            else:
                local_beta_u[user] += (float(rating) - alpha_4)/len(train_user_diction[user])
        else:
            if user not in local_beta_u:
                local_beta_u[user] = (float(rating) - alpha_4 - beta_i[item])/len(train_user_diction[user])
            else:
                local_beta_u[user] += (float(rating) - alpha_4 - beta_i[item])/len(train_user_diction[user])
    if epoch !=0 :
        delta = max(delta, abs(list(local_beta_u.values())[0] - list(beta_u.values())[0]))
    beta_u = local_beta_u
    
    
    for user, item, rating in data[:train_sample]:  # we first update beta_i
        if item not in local_beta_i:
            local_beta_i[item] = (float(rating) - alpha_4 - beta_u[user])/len(train_item_diction[item])
        else:
            local_beta_i[item] += (float(rating) - alpha_4 - beta_u[user])/len(train_item_diction[item])
    
    if epoch != 0:
        delta = max(delta, abs(list(local_beta_i.values())[0] - list(beta_i.values())[0]))
    
    beta_i = local_beta_i
    epoch += 1


train_prediction = np.zeros(train_sample)
test_prediction = np.zeros(total_sample - train_sample)
for index, (user, item, rating) in tqdm(enumerate(data[:train_sample]), total=train_sample):
    train_prediction[index] = alpha_4 + beta_u[user] + beta_i[item]

for index, (user, item, rating) in tqdm(enumerate(data[train_sample:]), total=total_sample-train_sample):
    if user in beta_u:
        if item in beta_i:
            test_prediction[index] = alpha_4 + beta_u[user]/2 + beta_i[item]/2
        else:
            test_prediction[index] = alpha_4 + beta_u[user]/2
    else:
        if item in beta_i:
            test_prediction[index] = alpha_4 + beta_i[item]/2
        else:
            test_prediction[index] = alpha_4
            
        
train_error_4 = mse(allRatings[:train_sample], train_prediction)
test_error_4 = mse(allRatings[train_sample:], test_prediction)
print('model 4 train mse', train_error_4)
print('model 4 train rmse', np.sqrt(train_error_4))
print('model 4 testing mse', test_error_4)
print('model 4 testing rmse', np.sqrt(test_error_4))

In [ ]:
train_user_diction[user]

In [ ]:
np.savetxt("Model4Output_1.csv",test_prediction)

In [ ]:
predictions = open(r"\Users\Julie\Documents\Andrew MBA stuff\Machine Learning\Kaggle Project\Rating\pairs_Rating.txt", 'w')
#reader = csv.reader(open(r"\Users\Julie\Documents\Andrew MBA stuff\Machine Learning\Kaggle Project\Rating\train_ratings.csv", 'r', encoding = "ISO-8859-1"), delimiter=',')


In [ ]:
total_sample = 14000
train_sample = 14000

allRatings = np.zeros(total_sample)   # store all ratings
test2_user_diction = {}   # store data using userID as keys
test2_item_diction = {}   # store data using itemID as keys
test_user_diction = {}   # store data using userID as keys
test_item_diction = {}   # store data using itemID as keys
data2 = np.zeros((total_sample, 3), dtype=object) # store entire data as an array


next(predictions) # to skip the first line

for index, row in enumerate(predictions):
    allRatings[index] = float(row[-1])
    data2[index] = row
    if index < train_sample:
        if row[0] not in test2_user_diction:
            test2_user_diction[row[0]] = [row[1:]]
        else:
            test2_user_diction[row[0]].append(row[1:])
        if row[1] not in test2_item_diction:
            test2_item_diction[row[1]] = [[row[0], row[2]]]
        else:
            test2_item_diction[row[1]].append([row[0], row[2]])
    else:
        if row[0] not in test_user_diction:
            test_user_diction[row[0]] = [row[1:]]
        else:
            test_user_diction[row[0]].append(row[1:])
        if row[1] not in test_item_diction:
            test_item_diction[row[1]] = [[row[0], row[2]]]
        else:
            test_item_diction[row[1]].append([row[0], row[2]])

## Matrix Factorization

In [ ]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(dataset, test_size=0.4, random_state = 1)

In [ ]:
import numpy as nd
test_u=list(test_data['reviewerID'])
test_i=list(test_data['itemID'])
test_r=list(test_data['rating'])
train_u=list(train_data['reviewerID'])
train_i=list(train_data['itemID'])
train_r=list(train_data['rating'])

In [ ]:
import d2l
from mxnet.gluon import nn
import mxnet as mx

import os
import pandas as pd
from mxnet import autograd, gluon, np, npx
npx.set_np()

In [ ]:
batch_size=1024
test_set = gluon.data.ArrayDataset(
    np.array(test_u), np.array(test_i), np.array(test_r))
train_set = gluon.data.ArrayDataset(
    np.array(train_u), np.array(train_i), np.array(train_r))
test_iter = gluon.data.DataLoader(
    test_set, batch_size=batch_size)
train_iter = gluon.data.DataLoader(
    train_set, shuffle=True, last_batch='rollover',
    batch_size=batch_size)

In [ ]:
data=pd.read_csv(r'\Users\Julie\Documents\Andrew MBA stuff\Machine Learning\Kaggle Project\Rating\train_ratings.csv')
data=data.rename(columns={"reviewerID":"user_id", "itemID":"isbn", "ratings":"rating"})
num_users = data.user_id.unique().shape[0]
num_items = data.isbn.unique().shape[0]

In [ ]:
class MF(nn.Block):
    def __init__(self, num_factors, num_users, num_items, **kwargs):
        super(MF, self).__init__(**kwargs)
        self.P = nn.Embedding(input_dim=num_users, output_dim=num_factors)
        self.Q = nn.Embedding(input_dim=num_items, output_dim=num_factors)
        self.user_bias = nn.Embedding(num_users, 1)
        self.item_bias = nn.Embedding(num_items, 1)

    def forward(self, user_id, isbn):
        P_u = self.P(user_id)
        Q_i = self.Q(isbn)
        b_u = self.user_bias(user_id)
        b_i = self.item_bias(isbn)
        outputs = (P_u * Q_i).sum(axis=1) + np.squeeze(b_i) + np.squeeze(b_u)
        return outputs.flatten()

In [ ]:
def evaluator(net, test_iter, ctx):
    rmse = mx.metric.RMSE()  # Get the RMSE
    rmse_list = []
    for idx, (users, items, ratings) in enumerate(test_iter):
        u = gluon.utils.split_and_load(users, ctx, even_split=False)
        i = gluon.utils.split_and_load(items, ctx, even_split=False)
        r_ui = gluon.utils.split_and_load(ratings, ctx, even_split=False)
        r_hat = [net(u, i) for u, i in zip(u, i)]
        rmse.update(labels=r_ui, preds=r_hat)
        rmse_list.append(rmse.get()[1])
    return float(np.mean(np.array(rmse_list)))

In [ ]:
# Saved in the d2l package for later use
def train_recsys_rating(net, train_iter, test_iter, loss, trainer, num_epochs,
                        ctx_list=d2l.try_all_gpus(), evaluator=None,
                        **kwargs):
    timer = d2l.Timer()
    animator = d2l.Animator(xlabel='epoch', xlim=[1, num_epochs], ylim=[0, 2],
                            legend=['train loss', 'test RMSE'])
    for epoch in range(num_epochs):
        metric, l = d2l.Accumulator(3), 0.
        for i, values in enumerate(train_iter):
            timer.start()
            input_data = []
            values = values if isinstance(values, list) else [values]
            for v in values:
                input_data.append(gluon.utils.split_and_load(v, ctx_list))
            train_feat = input_data[0:-1] if len(values) > 1 else input_data
            train_label = input_data[-1]
            with autograd.record():
                preds = [net(*t) for t in zip(*train_feat)]
                ls = [loss(p, s) for p, s in zip(preds, train_label)]
            [l.backward() for l in ls]
            l += sum([l.asnumpy() for l in ls]).mean() / len(ctx_list)
            trainer.step(values[0].shape[0])
            metric.add(l, values[0].shape[0], values[0].size)
            timer.stop()
        if len(kwargs) > 0:  # it will be used in section AutoRec.
            test_rmse = evaluator(net, test_iter, kwargs['inter_mat'],
                                  ctx_list)
        else:
            test_rmse = evaluator(net, test_iter, ctx_list)
        train_l = l / (i + 1)
        animator.add(epoch + 1, (train_l, test_rmse))
    print('train loss %.3f, test RMSE %.3f'
          % (metric[0] / metric[1], test_rmse**2))
    print('%.1f examples/sec on %s'
          % (metric[2] * num_epochs / timer.sum(), ctx_list))

In [ ]:
ctx = d2l.try_all_gpus()
num_users, num_items
net = MF(30, num_users, num_items)
net.initialize(ctx=ctx, force_reinit=True, init=mx.init.Xavier())
lr, num_epochs, wd, optimizer = 0.002, 10, .05, 'sgd'
loss = gluon.loss.L2Loss()
trainer = gluon.Trainer(net.collect_params(), optimizer,
                        {"learning_rate": lr, 'wd': wd})
train_recsys_rating(net, train_iter, test_iter, loss, trainer, num_epochs,
                    ctx, evaluator)

In [ ]:
data2=pd.read_csv(r'\Users\Julie\Documents\Andrew MBA stuff\Machine Learning\Kaggle Project\Rating\pairs_Rating.csv')